In [1]:
import pandas as pd
from datetime import date
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM,Activation
import tensorflow as tf

In [2]:
df = pd.read_csv('m-hsmort7112.txt', sep=' ')

In [4]:
df.head()

,year,mon,hs,mort
0,1971,4,1986,7.31
1,1971,5,2049,7.43
2,1971,6,2026,7.53
3,1971,7,2083,7.60
4,1971,8,2158,7.70


In [7]:
df['Date'] = df['year'].astype(str)+' /'+df['mon'].astype(str)

In [8]:
df_new = df[['Date', 'mort']]

In [13]:
train_values = df_new.mort.values.reshape(-1,1)
#train_values

In [15]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_dataset = scaler.fit_transform(train_values)

In [17]:
def create_dataset(scaled_data):
    window_size = 6
    x_set =[]
    y_set =[]
    
    for i in range(window_size,len(scaled_data)):
        x_set.append(scaled_data[i-window_size:i,:])
        y_set.append(scaled_data[i,0:1])
        
    return x_set, y_set

In [19]:
x_set, y_set = create_dataset(scaled_dataset)
x_data = np.array(x_set)
y_data = np.array(y_set)
print(x_data.shape)
print(y_data.shape)

(486, 6, 1)
(486, 1)


In [21]:
#Split data
#x_data
train_size = int(len(x_data)*0.8)
test_size_x = len(x_data) - train_size
train_data_x = x_data[:train_size]
test_data_x =  x_data[train_size:]

print(train_data_x.shape[1])

#y_data
train_size = int(len(y_data)*0.8)
test_size_y = len(y_data) - train_size
train_data_y = y_data[:train_size]
test_data_y =  y_data[train_size:]
print(train_data_y.shape)

6
(388, 1)


In [23]:
#LSTM model 
model = Sequential()

model.add(LSTM(units = 256, return_sequences = True, input_shape=(train_data_x.shape[1], train_data_x.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units = 256, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 256, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 256))
model.add(Dropout(0.2))

model.add(Dense(units = 1))


model.compile(optimizer = 'adam', loss = 'mean_squared_error')   

In [25]:
history = model.fit(train_data_x, train_data_y, epochs = 200, batch_size=80)

Epoch 1/200
5/5 [==============================] - 1s 147ms/step - loss: 0.0011
Epoch 2/200
5/5 [==============================] - 1s 155ms/step - loss: 0.0011
Epoch 3/200
5/5 [==============================] - 1s 145ms/step - loss: 8.2543e-04
Epoch 4/200
5/5 [==============================] - 1s 149ms/step - loss: 0.0011
Epoch 5/200
5/5 [==============================] - 1s 156ms/step - loss: 0.0013
Epoch 6/200
5/5 [==============================] - 1s 144ms/step - loss: 0.0012
Epoch 7/200
5/5 [==============================] - 1s 150ms/step - loss: 9.8518e-04
Epoch 8/200
5/5 [==============================] - 1s 156ms/step - loss: 9.0390e-04
Epoch 9/200
5/5 [==============================] - 1s 155ms/step - loss: 9.4821e-04
Epoch 10/200
5/5 [==============================] - 1s 142ms/step - loss: 0.0010
Epoch 11/200
5/5 [==============================] - 1s 145ms/step - loss: 0.0012
Epoch 12/200
5/5 [==============================] - 1s 168ms/step - loss: 0.0010
Epoch 13/200
5/5 [===

5/5 [==============================] - 1s 210ms/step - loss: 7.9801e-04
Epoch 194/200
5/5 [==============================] - 1s 182ms/step - loss: 5.7035e-04
Epoch 195/200
5/5 [==============================] - 1s 188ms/step - loss: 6.4493e-04
Epoch 196/200
5/5 [==============================] - 1s 172ms/step - loss: 7.5450e-04
Epoch 197/200
5/5 [==============================] - 1s 173ms/step - loss: 6.7319e-04
Epoch 198/200
5/5 [==============================] - 1s 170ms/step - loss: 7.7748e-04
Epoch 199/200
5/5 [==============================] - 1s 175ms/step - loss: 7.5810e-04
Epoch 200/200
5/5 [==============================] - 1s 171ms/step - loss: 7.5564e-04


In [27]:
y_predicted = model.predict(test_data_x)
y_predicted.shape

4/4 [==============================] - 0s 24ms/step


(98, 1)

In [28]:
#Calculate RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(scaler.inverse_transform(test_data_y), scaler.inverse_transform(y_predicted))

rmse = np.sqrt(mse)
rmse

0.19922234349383686